In [1]:
import pandas as pd

# Load

In [2]:
train = pd.read_csv('../input/application_train.csv.zip')
test  = pd.read_csv('../input/application_test.csv.zip')

trte = pd.concat([train, test], ignore_index=True)

prev = pd.read_csv('../input/previous_application.csv.zip')
prev = pd.merge(prev, trte[['SK_ID_CURR', 'CODE_GENDER']], on='SK_ID_CURR')


keys = ['AMT_ANNUITY',
        'AMT_CREDIT',
        'AMT_GOODS_PRICE',
        'NAME_CONTRACT_TYPE',
        'WEEKDAY_APPR_PROCESS_START',
        'CODE_GENDER',
        'HOUR_APPR_PROCESS_START',
        'NAME_TYPE_SUITE']

prev_test = pd.merge(prev, test.drop_duplicates(keys, False), 
                     on=keys, how='inner')

# current best
sub = pd.read_csv('../data/stochastic_blending_v3-3_0.411rk7_0.304mw1_0.285t1_0.81031CV_0.3Adv.csv')

pos = pd.read_csv('../input/POS_CASH_balance.csv.zip')


# If DPD mean == 0, TARGET = 0?

In [3]:
leak_ids = prev_test.SK_ID_CURR_y.value_counts()
leak_ids_test = leak_ids[leak_ids==1].index

In [4]:
df = prev_test[prev_test.SK_ID_CURR_y.isin(leak_ids_test)][['SK_ID_PREV', 'SK_ID_CURR_x', 'SK_ID_CURR_y']].sort_values('SK_ID_PREV')
leak_ids_prev = df.SK_ID_PREV

In [5]:
pos[pos.SK_ID_CURR.isin(leak_ids_test)].shape, pos[pos.SK_ID_PREV.isin(leak_ids_prev)].shape

((10397, 8), (2072, 8))

In [6]:
pos_ = pos[pos.SK_ID_PREV.isin(leak_ids_prev)]

dpd_mean = pos_.groupby('SK_ID_PREV').SK_DPD.mean()

In [7]:
df.set_index('SK_ID_PREV', inplace=True)

In [8]:
df['dpd_mean'] = dpd_mean

In [9]:
df.dpd_mean.describe()

count    161.000000
mean       0.063685
std        0.334841
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.380952
Name: dpd_mean, dtype: float64

In [10]:
ids_0 = df[df.dpd_mean==0].SK_ID_CURR_y.tolist()
ids_1 = df[df.dpd_mean>0].SK_ID_CURR_y.tolist()

In [11]:
sub.sort_values('TARGET', inplace=True)

sub_0 = sub[sub.SK_ID_CURR.isin(ids_0)].reset_index(drop=True)
sub_1 = sub[sub.SK_ID_CURR.isin(ids_1)].reset_index(drop=True)
sub_other = sub[~sub.SK_ID_CURR.isin(ids_0+ids_1)].reset_index(drop=True)

In [12]:
sub_0

,SK_ID_CURR,TARGET
0,102293,0.006073
1,277290,0.013971
2,108763,0.016002
3,318857,0.018854
4,324995,0.022567
5,170203,0.022608
6,214141,0.025275
7,347097,0.037564
8,430467,0.040969
9,252092,0.041708


In [13]:
sub_1

,SK_ID_CURR,TARGET
0,367663,0.007119
1,122257,0.034897
2,137590,0.107029
3,447230,0.184453
4,193198,0.238245
5,219817,0.272546
6,442195,0.414923
7,163895,0.615378
8,352451,0.905753
9,311024,0.912912


# Swap

In [14]:
sub_0.shape, sub_1.shape

((150, 2), (11, 2))

In [15]:
sub_0.TARGET.describe()

count    150.000000
mean       0.432215
std        0.282045
min        0.006073
25%        0.205825
50%        0.389217
75%        0.639494
max        0.960918
Name: TARGET, dtype: float64

In [16]:
sub_1.TARGET.describe()

count    11.000000
mean      0.425328
std       0.369244
min       0.007119
25%       0.145741
50%       0.272546
75%       0.760565
max       0.985352
Name: TARGET, dtype: float64

In [17]:
target_all = sorted(sub_0.append(sub_1).TARGET.values)

In [18]:
target_0, target_1 = target_all[:sub_0.shape[0]], target_all[sub_0.shape[0]:]

In [19]:
len(target_0), len(target_1)

(150, 11)

In [20]:
sub_0.TARGET = target_0
sub_1.TARGET = target_1

In [21]:
sub_0

,SK_ID_CURR,TARGET
0,102293,0.006073
1,277290,0.007119
2,108763,0.013971
3,318857,0.016002
4,324995,0.018854
5,170203,0.022567
6,214141,0.022608
7,347097,0.025275
8,430467,0.034897
9,252092,0.037564


In [22]:
sub_1

,SK_ID_CURR,TARGET
0,367663,0.914307
1,122257,0.919149
2,137590,0.923334
3,447230,0.932320
4,193198,0.948814
5,219817,0.949573
6,442195,0.952651
7,163895,0.954087
8,352451,0.956959
9,311024,0.960918


In [23]:
sub_new = pd.concat([sub_0, sub_1, sub_other])
sub_new.sort_values('TARGET', inplace=True)
sub_new.reset_index(drop=True, inplace=True)

sub_new.shape

(48744, 2)

In [24]:
sub.shape

(48744, 2)

In [25]:
sub_new.SK_ID_CURR.unique().shape

(48744,)

In [26]:
sub_new.to_csv('../output/stochastic_blending_v3-3_0.411rk7_0.304mw1_0.285t1_0.81031CV_0.3Adv_Leak002.csv.gz', index=False, compression='gzip')